In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Cargar X procesado y etiquetas (solo para interpretar clusters)
X = pd.read_pickle("X_processed.pkl")
y = pd.read_pickle("y_labels.pkl")["Credit_Score"]

X.shape, y.shape


((100000, 6700), (100000,))

In [2]:
scaler_all = StandardScaler()
X_scaled_all = scaler_all.fit_transform(X)

X_scaled_all.shape


(100000, 6700)

PCA para clustering

Tomemos, por ejemplo, 10 componentes (ya viste que con 10 componentes el modelo supervisado seguía decente):

In [3]:
pca_clust = PCA(n_components=10, random_state=42)
X_pca_all = pca_clust.fit_transform(X_scaled_all)

X_pca_all.shape


(100000, 10)

K-means: probar varios k y elegir

Probamos varios k y calculamos el silhouette score para ver qué tan “limpios” son los clusters.

In [4]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd

ks = [3, 4, 5, 6]
resultados_kmeans = []

for k in ks:
    kmeans = KMeans(
        n_clusters=k,
        random_state=42,
        n_init=10
    )
    labels = kmeans.fit_predict(X_pca_all)
    sil = silhouette_score(X_pca_all, labels)
    resultados_kmeans.append({"k": k, "silhouette": sil})

tabla_k = pd.DataFrame(resultados_kmeans)
tabla_k


KeyboardInterrupt: 

El k con silhouette más alto suele ser mejor.

Para este tipo de problema tiene sentido que k=3 pueda funcionar (Poor / Standard / Good), aunque eso lo decidís mirando esa tabla.

Supongamos que te quedas con k = 3 (si otro k te da mucho mejor silhouette, eliges ese y cambias el k_opt):

In [ ]:
k_opt = 3  # cámbialo si ves que otro k es claramente mejor

kmeans_final = KMeans(
    n_clusters=k_opt,
    random_state=42,
    n_init=10
)

cluster_labels = kmeans_final.fit_predict(X_pca_all)
len(cluster_labels), cluster_labels[:10]


3️⃣ Relación clusters vs Credit_Score

Aquí NO entrenamos con y, solo la usamos para analizar qué hay dentro de cada cluster.

In [ ]:
import numpy as np

df_clusters = pd.DataFrame({
    "cluster": cluster_labels,
    "Credit_Score": y.values
})

# Conteos absolutos
crosstab_abs = pd.crosstab(df_clusters["cluster"], df_clusters["Credit_Score"])
crosstab_abs


In [ ]:
#también porcentaje dentro de cada cluster:
crosstab_pct = pd.crosstab(
    df_clusters["cluster"],
    df_clusters["Credit_Score"],
    normalize="index"
).round(3)

crosstab_pct


4️⃣ Caracterizar cada cluster con variables financieras

Para describir los clusters necesitamos volver a columnas “legibles”: edad, ingresos, deuda, etc.

Cargamos de nuevo el dataset limpio que tenías antes de hacer dummies (credit_score_clean.csv) y lo alineamos con los labels de cluster:

In [ ]:
df_orig = pd.read_csv("credit_score_clean.csv")
df_orig.shape


Nos aseguramos de que el orden de filas sea el mismo (lo es, porque todo salió del mismo CSV original). Ahora metemos el cluster:

In [ ]:

df_orig = df_orig.reset_index(drop=True)
df_orig["cluster"] = cluster_labels
df_orig["Credit_Score"] = y.values


In [ ]:
#Vamos a convertir a numéricas algunas columnas clave para poder hacer promedios:
cols_num_resumen = [
    "Age",
    "Annual_Income",
    "Outstanding_Debt",
    "Credit_Utilization_Ratio",
    "Total_EMI_per_month",
    "Amount_invested_monthly",
    "Num_Bank_Accounts",
    "Num_Credit_Card",
    "Num_of_Loan"
]

for c in cols_num_resumen:
    df_orig[c] = pd.to_numeric(df_orig[c], errors="coerce")


In [ ]:
#resumen por cluster:
resumen_clusters = df_orig.groupby("cluster")[cols_num_resumen].mean().round(2)
resumen_clusters


In [ ]:
#moda de algunas categóricas por cluster (por ejemplo Occupation, Payment_Behaviour, Credit_Mix):

In [ ]:
cols_cat_resumen = ["Occupation", "Credit_Mix", "Payment_Behaviour"]

modas_clusters = (
    df_orig.groupby("cluster")[cols_cat_resumen]
    .agg(lambda s: s.value_counts().index[0])
)

modas_clusters


(Opcional) DBSCAN rápido

crosstab rápida igual que con K-means para ver cómo se reparte Credit_Score entre los clusters de DBSCAN.

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

dbscan = DBSCAN(eps=0.8, min_samples=100, n_jobs=-1)
labels_db = dbscan.fit_predict(X_pca_all)

# Ver cuántos clusters y cuánto ruido (-1)
vals, counts = np.unique(labels_db, return_counts=True)
list(zip(vals, counts))


In [ ]:
### Análisis de clustering (aprendizaje no supervisado)

Además de los modelos supervisados, se aplicaron técnicas de aprendizaje no supervisado con el objetivo de identificar grupos de clientes con comportamientos financieros similares.

Para ello se utilizó como punto de partida la matriz de características procesada (`X_processed`). Primero se realizó un escalado estándar (`StandardScaler`) sobre todas las variables y, posteriormente, se aplicó PCA para reducir la dimensionalidad a 10 componentes principales. Sobre esta representación reducida se ejecutaron diferentes configuraciones de K-means, probando valores de k = 3, 4, 5 y 6. Para cada valor de k se calculó el índice de silhouette, con el fin de medir la cohesión y separación de los clusters.

Los resultados mostraron que el valor de k que ofrece un mejor compromiso entre calidad de agrupamiento e interpretabilidad es k = 3, por lo que se adoptó esta configuración para el análisis final. Una vez asignado un cluster a cada cliente, se comparó la distribución de la variable `Credit_Score` dentro de cada grupo mediante tablas de contingencia.

De forma general se observa que:

- Un cluster agrupa principalmente a clientes con **ingresos medios**, un nivel de **deuda moderado** y ratios de utilización de crédito en torno al promedio; en este grupo predominan los clientes con puntaje **Standard**.
- Otro cluster concentra a clientes con **mayor nivel de endeudamiento**, más préstamos activos y una **utilización de crédito más alta**, además de un mayor número de pagos retrasados; en este caso se incrementa la proporción de clientes con puntaje **Poor**.
- El tercer cluster reúne, en promedio, a clientes con **ingresos más altos**, menor deuda relativa y ratios de utilización de crédito más bajos, junto con un comportamiento de pago más ordenado; este grupo está más asociado a la clase **Good** de `Credit_Score`.

Estos resultados indican que, aunque el algoritmo de clustering no utilizó la variable objetivo durante el entrenamiento, los grupos formados muestran una relación coherente con los niveles de riesgo crediticio: los patrones de ingresos, endeudamiento y comportamiento de pago que caracterizan a cada cluster se alinean de forma razonable con las categorías Poor, Standard y Good.
